In [5]:
import datetime
import time
import psycopg2

url = "hdfs://127.0.0.1:9000/socmed_input/"
out_url = "hdfs://127.0.0.1:9000/socmed_output/"

conn = psycopg2.connect(database="bigdatatest",
    host="localhost",
    user="geraldakbar",
    password='bigdata'
)

cur = conn.cursor()

In [3]:
def normalizeTime(time):
    res = ''
    for i in range(len(time)):
        if i == 6 or i == 7:
            res += '0'
        elif i == 4:
            if int(time[i]) < 5:
                res += '0'
            else:
                res += '5'
        else:
            res += time[i] 
    return res

def parseTime(type, time):
    if type == "twitter":
        ts = t.strftime('%Y-%m-%d %H:%M:%S', t.strptime(time,'%a %b %d %H:%M:%S +0000 %Y'))
        return ts[:10]
    elif type == "instagram" or type == "other":
        time = str(datetime.datetime.fromtimestamp(int(time)).date())
        return time[:10]
    else:
        return time[:10] + ' ' + normalizeTime(time[11:19])

In [7]:
twt = spark.read.json(f"{url}twitter*.json").rdd
# map the lines to a key-value pair
def map(line):
    date = line['created_at']
    # format date
    date = datetime.datetime.strptime(date, '%a %b %d %H:%M:%S %z %Y').strftime('%Y-%m-%d')
    return (("twitter", date), 1)
def userMap(line):
    user = line['user_id']
    date = line['created_at']
    # format date
    date = datetime.datetime.strptime(date, '%a %b %d %H:%M:%S %z %Y').strftime('%Y-%m-%d')
    return (("twitter", date, user), 1)

twtPost = twt.map(map)
twtPost = twtPost.reduceByKey(lambda x, y: x + y)

user = twt.map(userMap).distinct().map(lambda x: ((x[0][0], x[0][1]), x[1]))
user = user.reduceByKey(lambda x, y: x + y)

twtPost = twtPost.join(user).map(lambda x: (x[0], x[1][0], x[1][1]))
twtPost = twtPost.map(lambda x: (x[0][0], x[0][1], x[1], x[2]))

test = twtPost.take(10)
insertToSQL(test,conn,cur)

In [1]:
def insertToSQL(data,connection,cursor):
    for element in data:
        social_media = element[0]
        timestamp = element[1]
        count = element[2]
        uniquecount = element[3]

        query = f"""INSERT INTO social_media_data (social_media, timestamp, count, uniquecount)
                    VALUES ('{social_media}', '{timestamp}', {count}, {uniquecount})
                    ON CONFLICT (social_media, timestamp) DO UPDATE
                    SET count = CASE WHEN social_media_data.count IS NULL THEN 1 ELSE social_media_data.count + {count} END,
                    uniquecount = CASE WHEN social_media_data.uniquecount IS NULL THEN 1 ELSE social_media_data.uniquecount + {uniquecount} END,
                    updated_at = NOW();"""



        cursor.execute(query)

    connection.commit()